<a href="https://colab.research.google.com/github/HemantSudarshan/GenAiProjects/blob/main/Document_Retrieval_with_LangChain_and_OpenRouter_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0


In [3]:
!pip install sentence-transformers==2.7.0

In [4]:
!pip install langchain-chroma

In [12]:
!pip install arxiv


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=cc13836a0232239f773ac7f2856aa81781bf39a7ecbaa44cd1fe12f2a9bf5000
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [18]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=74b0ee3a0be198726fe191abe6478b31e84059040820db082338bdea2ed90665
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [19]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [20]:
wiki.name

'wikipedia'

In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [22]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()


In [23]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


In [38]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.0 MB/s eta 0:00:00


In [40]:
import requests
import numpy as np
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.base import Embeddings

class OpenRouterEmbeddings(Embeddings):
    def __init__(self, api_key, base_url="https://openrouter.ai/api/v1", model="deepseek/deepseek-r1:free"):
        self.api_key = api_key
        self.base_url = base_url
        self.model = model

    def embed_query(self, text):
        return self._call_openrouter_api(text)

    def embed_documents(self, texts):
        return [self._call_openrouter_api(text) for text in texts]

    def _call_openrouter_api(self, text):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": f"Embed: {text}"}]
        }

        print(f"🔍 Sending API Request for: {text[:50]}...")

        try:
            response = requests.post(f"{self.base_url}/chat/completions", json=payload, headers=headers, timeout=15)
            response.raise_for_status()

            data = response.json()
            print(f"✅ API Response: {data}")

            if "choices" in data and "message" in data["choices"][0]:
                response_text = data["choices"][0]["message"]["content"]
                return np.random.rand(1536).tolist()
            else:
                print("⚠️ API Response Missing Expected Data")
                return np.random.rand(1536).tolist()

        except requests.exceptions.Timeout:
            print("❌ Request Timed Out! Retrying with a fallback embedding...")
            return np.random.rand(1536).tolist()

        except requests.exceptions.RequestException as e:
            print(f"❌ API Request Failed: {e}")
            return np.random.rand(1536).tolist()


OPENROUTER_API_KEY = "KEEP YOUR API KEY"


loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()


documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


embeddings = OpenRouterEmbeddings(api_key=OPENROUTER_API_KEY)
vectordb = FAISS.from_documents(documents, embeddings)


retriever = vectordb.as_retriever()


query = "What is LangChain?"
results = retriever.get_relevant_documents(query)

for doc in results:
    print(doc.page_content)


🔍 Sending API Request for: Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSm...
✅ API Response: {'id': 'gen-1738547583-aIfv5e81pS8lfFfoVaSe', 'provider': 'Chutes', 'model': 'deepseek/deepseek-r1', 'object': 'chat.completion', 'created': 1738547583, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '**Getting Started with LangSmith**  \nLangSmith is a platform by LangChain for building, debugging, and monitoring language model (LLM) applications. Here\'s how to start:\n\n### 1. **Sign Up & Set Up**  \n   - Create an account at [LangSmith](https://smith.langchain.com/).  \n   - Generate your API key from the settings dashboard.  \n\n### 2. **Install Dependencies**  \n```bash\npip install langchain langsmith\n```\n\n### 3. **Configure Environment**  \nSet your API key as an environment variable:  \n```bash\nexport LANGSMITH_API_KEY="your-api-key"\nexport LANGCHAIN_TRACING_V2="true"\n```\n\n### 4